# FastAPI for fun and profit

FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.6+ based on standard Python type hints.

## Features

 - Fast: Very high performance, on par with NodeJS and Go (thanks to Starlette and Pydantic).
 - One of the fastest Python frameworks available.
 - Easy to use, with great developer ergonimics
 - Excellent editor support
 - Robust and standards-based
 - OpenAPI and JSON Schema support (and good support, too)
 - Open Source (MIT)

## A Brief discussion of types

In [1]:
def hello_with_age(person_to_greet, age_in_years):
    in_100_years = age_in_years + 100
    return f'Hello {person_to_greet} in 100 years you will be {in_100_years}'

In [2]:
hello_with_age('PGH Python Meetup', '6')

TypeError: can only concatenate str (not "int") to str

In [3]:
def hello_with_age(person_to_greet: str, age_in_years: int) -> str:
    in_100_years = age_in_years + 100
    return f'Hello {person_to_greet} in 100 years you will be {in_100_years}'

In [4]:
hello_with_age('PGH Python Meetup', 6)

'Hello PGH Python Meetup in 100 years you will be 106'

### This is important because of Pydantic

Pydantic is a data validation library that you'll want to use with FastAPI.  It's great for defining the shape of your objects (it works with dataclasses, too).  Pydantic's `BaseModel` also provides nice serialization.

In [5]:
from pydantic import BaseModel

class Helloer(BaseModel):
    person_to_greet: str
    age_in_years: int
        
    def greet(self):
        in_100_years = self.age_in_years * 100
        return f'Hello {self.person_to_greet} in 100 years you will be {in_100_years}'

def hello_with_pydantic(person: Helloer) -> str:
    return person.greet()    

In [6]:
Helloer(person_to_greet='Josh', age_in_years=10)

<Helloer person_to_greet='Josh' age_in_years=10>

In [7]:
Helloer(**{'person_to_greet': 'Josh', 'age_in_years': '10'})

<Helloer person_to_greet='Josh' age_in_years=10>

In [8]:
Helloer(person_to_greet='Josh')

ValidationError: 1 validation error for Helloer
age_in_years
  field required (type=value_error.missing)

# Pydantic can do all manner of validations

In [9]:
from pydantic import BaseModel, UrlStr, ValidationError, validator
from pydantic.color import Color
from enum import IntEnum

class HLevel(IntEnum):
    para = 0
    header = 1
    subhead = 2
    subsubhead = 3  

class HtmlThingy(BaseModel):
    level: HLevel
    text: str
    color: Color
    site: UrlStr
        
    @validator('text')
    def text_length(cls, v, values, **kwargs):
        if 'level' in values and values['level'] in {HLevel.header, HLevel.subhead}:
            if len(v) > 12:
                raise ValueError('Text is too long for level')
        return v

In [10]:
HtmlThingy(level=HLevel.para,
           text="I'm a little teapot",
           color='#000000',
           site='http://gridwise.com')

<HtmlThingy level=<HLevel.para: 0> text="I'm a little teapot" color=<Color('black', (0, 0, 0))> site='http://gridwise.com'>

In [11]:
HtmlThingy(level=HLevel.header,
           text="I'm a little teapot",
           color='#000000',
           site='http://gridwise.com')

ValidationError: 1 validation error for HtmlThingy
text
  Text is too long for level (type=value_error)

In [12]:
HtmlThingy(level=HLevel.header,
           text="Teapots!",
           color='#000000',
           site='http:gridwise.com')

ValidationError: 1 validation error for HtmlThingy
site
  url string does not match regex (type=value_error.url.regex)

In [13]:
HtmlThingy(level=HLevel.header,
           text="Teapots!",
           color='lightblue',
           site='http://gridwise.com')

<HtmlThingy level=<HLevel.header: 1> text='Teapots!' color=<Color('lightblue', (173, 216, 230))> site='http://gridwise.com'>

# Pydantic can serialize and deserialize

In [14]:
HtmlThingy(level=HLevel.header,
           text="Teapots!",
           color='lightblue',
           site='http://gridwise.com').dict()

{'level': <HLevel.header: 1>,
 'text': 'Teapots!',
 'color': <Color('lightblue', (173, 216, 230))>,
 'site': 'http://gridwise.com'}

In [15]:
HtmlThingy(level=HLevel.header,
           text="Teapots!",
           color='lightblue',
           site='http://gridwise.com').json()

'{"level": 1, "text": "Teapots!", "color": "lightblue", "site": "http://gridwise.com"}'

In [16]:
import json

HtmlThingy(**json.loads(_))

<HtmlThingy level=<HLevel.header: 1> text='Teapots!' color=<Color('lightblue', (173, 216, 230))> site='http://gridwise.com'>

# Hello FastAPI

In [17]:
%%writefile hello.py
from fastapi import FastAPI
from pydantic import BaseModel


app = FastAPI()

class Greeting(BaseModel):
    message: str

@app.get("/", response_model=Greeting)
async def root(name: str = None):
    if name is None:
        return Greeting(message="Hello World")
    else:
        return Greeting(message=f"Hello {name}")

Overwriting hello.py


# run with uvicorn

```$ uvicorn hello:app```

In [18]:
import requests

requests.get('http://localhost:8000').json()

{'message': 'Hello World'}

In [19]:
requests.get('http://localhost:8000',params={'name': 'PGH Python'}).json()

{'message': 'Hello PGH Python'}

This can be run using `uvicorn hello:app --reload` and then tested with `ab` giving 1838 (ish) requests per second

In [20]:
%%writefile sleepy.py
from fastapi import FastAPI
from pydantic import BaseModel
import time
import random

random.seed(42)

app = FastAPI()

class Greeting(BaseModel):
    message: str

@app.get("/", response_model=Greeting)
async def root(name: str = None):
    if random.randint(0,100) <= 10:
        time.sleep(0.5)
    if name is None:
        return Greeting(message="Hello World")
    else:
        return Greeting(message=f"Hello {name}")

Overwriting sleepy.py


In [21]:
requests.get('http://localhost:8000',
             params={'badname': 'PGH Python'}).json()

{'message': 'Hello World'}

In [22]:
%%writefile sleepy.py
from fastapi import FastAPI, Query
from pydantic import BaseModel
import time
import random

random.seed(42)

app = FastAPI()

class Greeting(BaseModel):
    message: str

@app.get("/", response_model=Greeting)
async def root(name: str = Query(..., min_length=3)):
    if random.randint(0,100) <= 10:
        time.sleep(0.5)
    if name is None:
        return Greeting(message="Hello World")
    else:
        return Greeting(message=f"Hello {name}")

Overwriting sleepy.py


In [23]:
requests.get('http://localhost:8000',
             params={'badname': 'PGH Python'}).json()

{'detail': [{'loc': ['query', 'name'],
   'msg': 'field required',
   'type': 'value_error.missing'}]}

In [24]:
requests.get('http://localhost:8000',
             params={'name': 'PG'}).json()

{'detail': [{'loc': ['query', 'name'],
   'msg': 'ensure this value has at least 3 characters',
   'type': 'value_error.any_str.min_length',
   'ctx': {'limit_value': 3}}]}

## A Real Application:  Bus Times

Let's build a real application.  We're going to connect to the truetime api
and get some information about busses.

### Endpoints

This api is going to have two endpoints.

 - GET arrivaltimes/{stop_id}
 - POST approximatetimes
 
The first is for api integration and the second is for HTML forms. 

In [25]:
%%writefile truetime.py
import requests
from pydantic import BaseModel, Schema
from enum import Enum
from typing import List
from environs import Env

env = Env()
env.read_env()

class BusType(str, Enum):
    ARRIVAL='A'
    DEPARTURE='D'

class Prediction(BaseModel):
    typ: BusType
    stpnm: str = Schema(..., title="Stop Number",
                        description="The bus stop id")
    rt: str = Schema(..., title="Route",
                     description="The bus route ID")
    des: str = Schema(...,
                      title="Destination")
    prdctdn: str = Schema(...,
                          title="Prediction",
                          description=("A string description that "
                                       "is most often minutes but "
                                       "can be DUE or other descriptive text"))

class Prd(BaseModel):
    prd: List[Prediction] = Schema(...,
                                   title="Predictions",
                                   description="A list of Prediction objects")

Overwriting truetime.py


In [26]:
%%writefile truetime.py -a
def get_predictions(stop_id: str) -> Prd:
    url = "http://truetime.portauthority.org/bustime/api/v3/getpredictions"

    querystring = {"key": env('PAKEY'),
                   "format": "json",
                   "rtpidatafeed": "Port Authority Bus",
                   "stpid": stop_id}

    response = requests.get(url, params=querystring)
    if response.status_code == 200:
        jdat = response.json()
        res = Prd(**jdat['bustime-response'])
        return res
    else:
        raise ValueError(response)

Appending to truetime.py


In [27]:
%%writefile bustimes.py
from fastapi import FastAPI, Path, Form
from pydantic import BaseModel
from truetime import get_predictions, Prd
app = FastAPI()

@app.get('/arrivaltimes/{stop_id}', response_model=Prd)
def get_stop_prediction(stop_id: str = Path(...,
                                            title="The ID Number of the bus stop.")):
    return get_predictions(stop_id)

@app.post('/approximatetimes', response_model=Prd)
def get_predictions_for_form(stop_id: str = Form(...,
                                                 title="the ID Number of the bus stop")):
    return get_predictions(stop_id)

Overwriting bustimes.py


In [28]:
requests.get('http://localhost:8000/arrivaltimes/7129').json()

{'prd': [{'typ': 'A',
   'stpnm': 'FORBES AVE + BEECHWOOD',
   'rt': '61A',
   'des': 'Braddock Shopping Ctr',
   'prdctdn': 'DUE'},
  {'typ': 'A',
   'stpnm': 'FORBES AVE + BEECHWOOD',
   'rt': '61B',
   'des': 'Braddock Hills Shopping Ctr',
   'prdctdn': '12'},
  {'typ': 'A',
   'stpnm': 'FORBES AVE + BEECHWOOD',
   'rt': '61A',
   'des': 'Braddock Shopping Ctr',
   'prdctdn': '14'},
  {'typ': 'A',
   'stpnm': 'FORBES AVE + BEECHWOOD',
   'rt': '61B',
   'des': 'Braddock',
   'prdctdn': '24'}]}

In [29]:
requests.post('http://localhost:8000/approximatetimes', data={'stop_id': '7129'}).json()

{'prd': [{'typ': 'A',
   'stpnm': 'FORBES AVE + BEECHWOOD',
   'rt': '61B',
   'des': 'Braddock Hills Shopping Ctr',
   'prdctdn': '11'},
  {'typ': 'A',
   'stpnm': 'FORBES AVE + BEECHWOOD',
   'rt': '61A',
   'des': 'Braddock Shopping Ctr',
   'prdctdn': '14'},
  {'typ': 'A',
   'stpnm': 'FORBES AVE + BEECHWOOD',
   'rt': '61B',
   'des': 'Braddock',
   'prdctdn': '24'}]}

## Testing

FastAPI tests well with pytest.  

In [30]:
%%writefile test_busapi.py
from starlette.testclient import TestClient
from bustimes import app

client = TestClient(app)


def test_read_main():
    response = client.get("/")
    assert response.status_code == 404

def test_get_prediction():
    response = client.get('/arrivaltimes/7129')
    assert response.status_code == 200
    
def test_html_form_prediction():
    response = client.post('/approximatetimes')
    assert response.status_code == 422
    
    response = client.post('/approximatetimes',
                           data={'stop_id': '7129'})
    assert response.status_code == 200

Overwriting test_busapi.py


In [31]:
!pytest test_busapi.py

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.1.2, py-1.8.0, pluggy-0.13.0
rootdir: /home/jsmith/Projects/personal/presentations/fast_api_presentation
collected 3 items                                                              

test_busapi.py ...                                                       [100%]

============================== 3 passed in 0.38s ===============================


## Now lets add a feature

I'd like to be able to POST json to the api, too.  We can easily add another endpoint
to take a json POST and then update the tests.  

In [32]:
%%writefile bustimes.py
from fastapi import FastAPI, Path, Form
from pydantic import BaseModel, Schema
from truetime import get_predictions, Prd
app = FastAPI()

class ArrivalRequest(BaseModel):
    stop_number: str = Schema(...,
                              description="the string version of the bus stop number")

@app.get('/arrivaltimes/{stop_id}', response_model=Prd)
def get_stop_prediction(stop_id: str = Path(...,
                                            title="The ID Number of the bus stop.")):
    return get_predictions(stop_id)

@app.post('/approximatetimes', response_model=Prd)
def get_predictions_for_form(stop_id: str = Form(...,
                                                 title="the ID Number of the bus stop")):
    return get_predictions(stop_id)

@app.post('/arrivaltimes', response_model=Prd)
def via_post_arrival_times(data: ArrivalRequest):
    return get_predictions(data.stop_number)

Overwriting bustimes.py


In [33]:
%%writefile test_busapi.py
from starlette.testclient import TestClient
from bustimes import app, ArrivalRequest

client = TestClient(app)


def test_read_main():
    response = client.get("/")
    assert response.status_code == 404

def test_get_prediction():
    response = client.get('/arrivaltimes/7129')
    assert response.status_code == 200
    
def test_html_form_prediction():
    response = client.post('/approximatetimes')
    assert response.status_code == 422
    
    response = client.post('/approximatetimes', data={'stop_id': '7129'})
    assert response.status_code == 200
    
def test_json_request():
    req = ArrivalRequest(stop_number="7129")
    response = client.post('/arrivaltimes', json=req.dict())
    assert response.status_code == 200

Overwriting test_busapi.py


In [34]:
!pytest test_busapi.py

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.1.2, py-1.8.0, pluggy-0.13.0
rootdir: /home/jsmith/Projects/personal/presentations/fast_api_presentation
collected 4 items                                                              

test_busapi.py ....                                                      [100%]

============================== 4 passed in 0.49s ===============================


# And Then There Are Docs

http://localhost:8000/redoc

http://localhost:8000/

# Thanks!


You can find this notebook at:

https://github.com/kognate/presentations/blob/master/fast_api_presentation/server.ipynb

I'm Josh Smith   
https://keybase.io/kognate
kognate@gmail.com